## Utilisation dans Azure d'un dataset

In [1]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# le data set est d'abord créé dans le menu Asset / Data / Create
# Actifs multimedias / Données / Créer (en français)

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("titanic_data", version="1")

df_original = pd.read_csv(data_asset.path)
df_original.head()

FileNotFoundError: [Errno 2] No such file or directory: 'null/Users/ncassonet.ext'

## Nouvelle version du dataset

In [3]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

df_cleaned = df_original.drop(columns=["Cabin"], errors="ignore")

# Sauvegarde locale du nouveau CSV
cleaned_path = "titanic_data_cleaned.csv"
df_cleaned.to_csv(cleaned_path, index=False)

new_version = ml_client.data.get("titanic_data", version="2")
if new_version :
    print(f"titanic_data version 2 allready exists")

else : 
    # Création de la nouvelle version du Data Asset (v2)
    new_data_asset = Data(
        path=cleaned_path,
        type=AssetTypes.URI_FILE,
        name="titanic_data",  # même nom
        version="2",              # nouvelle version
        description="Version nettoyée sans la colonne Cabin"
    )

    ml_client.data.create_or_update(new_data_asset)
    print(f"titanic_data version 2 created")


titanic_data version 2 allready exists


## Entraîner un modèle et consigner les résultats obtenus avec MLflow dans Azure ML

In [3]:
import pandas as pd
import mlflow
import mlflow.sklearn

# ajouté pour enregistrer le modèle
from mlflow import register_model

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential




# Connexion + chargement du dataset v2
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get(name="titanic_data_set", version="2")
df = pd.read_csv(data_asset.path)

# Préparation des données (exemple simple)
df = df.dropna()
X = df.drop(columns=["Survived"])
y = df["Survived"]

# Dummy encoding si besoin (ex: pour les colonnes catégorielles)
X = pd.get_dummies(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Démarrer un run MLflow
with mlflow.start_run()as running_mlflow :
    log_reg_titanic_model = LogisticRegression(max_iter=200)
    log_reg_titanic_model.fit(X_train, y_train)

    y_pred = log_reg_titanic_model.predict(X_test)
    y_proba = log_reg_titanic_model.predict_proba(X_test)[:, 1]

    # Métriques
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # Log avec MLflow
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("roc_auc", roc_auc)

    mlflow.sklearn.log_model(log_reg_titanic_model, "model")

    print(f"✅ Modèle loggé avec MLflow - F1: {f1:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, AUC: {roc_auc:.3f}")

    # Sauvegarde de l'ID du run
    run_id = running_mlflow.info.run_id
    print(f"Modèle loggé dans le run ID: {run_id}")

    registered_model = mlflow.register_model(
        model_uri=f"runs:/{run_id}/model",  # lien vers le modèle loggé
        name="log_reg_titanic_model"        # nom du modèle dans Azure ML
    )

    print(f"✅ Modèle enregistré dans Azure ML : {registered_model.name}, version : {registered_model.version}")
  

Found the config file in: /config.json
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/09 14:14:30 WARNING mlflow.models.model: Model

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.update_registered_model(
    name="log_reg_titanic_model",
    description="Modèle sklearn pour prédire la survie sur Titanic"
)

# ajout de description, tags
client.set_registered_model_tag("log_reg_titanic_model", "framework", "sklearn")